In [1]:
# Magic functions
%matplotlib
%load_ext autoreload
%autoreload 2



Using matplotlib backend: TkAgg


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [15]:
from calculate_zeta_t_lims import calculate_zeta_t_lims
from calculate_zeta_t_roots import calculate_zeta_t_roots
from find_marginalized_zeta_t_roots import find_marginalized_zeta_t_roots
import importlib
from log_C import log_C as log_C_func
import numpy as np
from plot_K_L import plot_K_L
import sys

In [ ]:



## Constants
lambs = (0, 0.5, 1.0)
zeta_sp_abs_lim = 5.0
zeta_sp_steps = 5000
n_pi = 4
K = np.exp(3)
ns = [100, 30, 10]

# zeta_sp mesh
zeta_sps = np.linspace(- zeta_sp_abs_lim, zeta_sp_abs_lim, zeta_sp_steps)
    
# Calculate roots
lambs_count = len(lambs)
ns_count = len(ns)
zeta_t_roots_size = (ns_count, lambs_count + 1, 2, zeta_sp_steps)
zeta_t_roots = np.zeros(zeta_t_roots_size, dtype = np.float)

for n_ind in range(ns_count):
    n = ns[n_ind]
    print("Processing n: %i/%i" % (n_ind + 1, ns_count))
    
    # Check if the requested K_L value can be reached
    n0 = n + n_pi
    log_C = log_C_func(n_0 = n0, n_pi = n_pi)

    if np.log(K) < log_C:
        raise ValueError("This value of the Bayes factor cannot be achieved for the current parameters")
    
    for zeta_ind in range(zeta_sp_steps):
        zeta_sp = zeta_sps[zeta_ind]

        # Fixed-lambda models
        for lamb_ind in range(lambs_count):
            lamb = lambs[lamb_ind]

            cur_roots = calculate_zeta_t_roots(zeta_sp, lamb, n, n_pi, K)
            # Save
            zeta_t_roots[n_ind, lamb_ind, 0, zeta_ind] = cur_roots[0]
            zeta_t_roots[n_ind, lamb_ind, 1, zeta_ind] = cur_roots[1] 

        # Marginalized
        cur_roots = find_marginalized_zeta_t_roots([zeta_sp], n, n_pi, K)
        # Save
        zeta_t_roots[n_ind, lambs_count, 0, zeta_ind] = cur_roots[0]
        zeta_t_roots[n_ind, lambs_count, 1, zeta_ind] = cur_roots[1]

print("Calculation terminated")
        
# print(zeta_t_roots)



In [7]:
# Plot
plot_K_L(zeta_sps, zeta_t_roots, ns)

NameError: name 'plot_K_L' is not defined

In [ ]:
from find_ito_roots import find_ito_roots
find_ito_roots(n_0, n, n_pi)

In [ ]:
from calculate_marginalized_integral import calculate_marginalized_integral
calculate_marginalized_integral([0.5], [1], 10, 4)

In [ ]:
y = [-1, 1]

In [ ]:
print (K)

In [31]:
from find_marginalized_zeta_sp_limit import find_marginalized_zeta_sp_limit

Ks = np.exp([-3, 0, 3])
n_max = 200
ns = range(1, n_max + 1)

Ks_count = len(Ks)

zeta_sp_borders = np.zeros((Ks_count, n_max), dtype = np.float)

# Calculate roots for different n and K
for K_ind in range(Ks_count):
    K = Ks[K_ind]
    for n_ind in range(n_max):
        n = ns[n_ind]
        zeta_sp_borders[K_ind, n_ind] = find_marginalized_zeta_sp_limit(n = n, n_pi = 4, K = K)
# print(zeta_sp_borders)


In [32]:
from plot_zeta_sp_borders import plot_zeta_sp_borders
plot_zeta_sp_borders(ns, zeta_sp_borders)